In [65]:
import gym

import clubs

import random

#GIT REPO WORKING

env = gym.make("PotLimitOmahaTwoPlayer-v0")

#print("PAST MAKING THE ENVIRONMENT")

#env.register_agents([clubs.agent.baselineOmahaAgent.BaselineOmahaAgent()] *2)
env.register_agents([clubs.agent.testingRLOmahaAgent.TestingRLOmahaAgent()] *2)

#print("AFTER REGISTERING AGENTS")

#print(env.observation_space)

print("observation space elements:")
for i in env.observation_space:
    print(i)


obs = env.reset()

print("--------------------------------------")
print("observations:")
print(obs)
print("--------------------------------------")
print(obs["hole_cards"])
print(obs["hole_cards"][0])
#print(get_rank_class(obs["hole_cards"][0]))


k = 10

for i in range(k):
    print(i)
    #print("in range of iterations")
    #print(obs)
    while True:
        #print("inside while loop")
        bet = env.act(obs)
        #print("just finished bet")
        obs, rewards, done, info = env.step(bet)
        #print("just finished step")

        playerNumber = obs["action"]
        #print("playernumber:", playerNumber)
        #if (playerNumber >= 0):
         #   print("within rewardUpdate step")
            #env.agents[playerNumber].sayHello(playerNumber)
        env.agents[playerNumber].setRewardandUpdate(obs,rewards[playerNumber])
        
        #print(obs)

        #for (i, agent) in env.agents.items():
        #    if obs["action"] == i: #obs[action] is the player
         #       print(obs)
         #       agent.sayHello(i)
        #print("-------------------------")
            #print(rewards[i])
            #agent.setRewardandUpdate(rewards[i]) #i give it
            

        #print(obs)
        #print("rewards: ", rewards)

        #print(env.agents[0])
        #env.agents[0].sayHello()

        if all(done):
            break

    #print(i)
    #print("Current stack",obs["stacks"])
    #print("------")
    if (i == k-1): #the last iteration
        print("Current stack",obs["stacks"])
    obs = env.reset()

print("Current stack",obs["stacks"])


observation space elements:
action
active
button
call
community_cards
hole_cards
max_raise
min_raise
pot
stacks
street_commits
--------------------------------------
observations:
{'action': 1, 'active': array([ True,  True]), 'button': 1, 'call': 1, 'community_cards': [], 'hole_cards': ['9♣', 'J♥', 'J♦', '2♠'], 'max_raise': 5, 'min_raise': 3, 'pot': 3, 'stacks': array([198, 199], dtype=int32), 'street_commits': array([2, 1], dtype=int32)}
--------------------------------------
['9♣', 'J♥', 'J♦', '2♠']
9♣
0
[0, 1, 2]
[1.00000000e+00 1.98000000e+02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00

KeyError: -1

In [121]:
#from . import base
from collections import defaultdict
import math
import numpy as np
from typing import Dict, List, Optional, Union

from clubs import error


STR_RANKS = "23456789TJQKA"
INT_RANKS = list(range(13))
RANK_DICT = dict(zip(list(STR_RANKS), INT_RANKS))

# pretty suits
PRETTY_SUITS: Dict[int, str] = {
    1: chr(9824),  # spades
    2: chr(9829),  # hearts
    4: chr(9830),  # diamonds
    8: chr(9827),  # clubs
}
    

In [73]:
#Hole Cards to Raise With
top30HandsList = [ ['A','A','K','K'], ['A','A','J','T'], ['A','A','Q','Q'], ['A','A','J','J'], ['A','A','T','T'],
                   ['A','A','9','9'], ['J','T','9','8'],
                   ['K','K','Q','Q'], ['K','K','J','J'], ['K','Q','J','T'], ['K','K','T','T'], ['K','K','A','Q'],
                   ['K','K','A','J'], ['K','K','A','T'], ['K','K','Q','J'], ['K','K','Q','T'], ['K','K','J','T'],
                   ['Q','Q','J','J'], ['Q','Q','T','T'], ['Q','Q','A','K'], ['Q','Q','A','J'], ['Q','Q','A','T'],
                   ['Q','Q','K','J'], ['Q','Q','K','T'], ['Q','Q','J','T'], ['Q','Q','J','9'], ['Q','Q','9','9'],
                   ['J','J','T','T'], ['J','J','T','9'] 
                 ]

high_face = ["T", "J", "Q", "K", "A"]
face = ["2","3","4","5","6","7","8","9", "T","J","Q","K","A"]

for i in range(len(face)):
    for j in range(len(face)):
        top30HandsList.append((["A", "A", face[i], face[j]]))

akxxs = []
for i in range(len(high_face)):
     for j in range(len(face)):
             akxxs.append((["A", "K", high_face[i], face[j]]))  
                
                
highFourInARowList = [ ['5','6','7','8'], ['6','7','8','9'], ['7','8','9','T'], ['8','9','T','J'], ['9','T','J','Q'],
                   ['T','J','Q','K'], ['J','Q','K','A']
                 ]

lowFourInARowList = [ ['4','5','6','7'], ['5','6','7','8'], ['6','7','8','9'], ['7','8','9','T'], ['8','9','T','J'], 
                     ['9','T','J','Q'],['T','J','Q','K'], ['J','Q','K','A'] ]

kkDoubleSuitedList = []
for i in range(len(face)):
    for j in range(len(face)):
        kkDoubleSuitedList.append((["K", "K", face[i], face[j]]))
        
        
# Hole Cards to Flop With
aqxxs = []
for i in range(len(high_face)):
     for j in range(len(face)):
             akxxs.append((["A", "Q", high_face[i], face[j]]))
                
axxxs = []
for i in range(len(face)):
    for j in range(len(face)):
        for k in range(len(face)):
            axxxs.append((["A",face[i],face[j],face[k]]))

In [74]:
#Initializing hole cards and rank and suit lists

holeCards = obs["hole_cards"]

rankList = []
suitList = []

for card in holeCards:
    rank_str = card.__str__()[0]
    rankList.append(rank_str)
    
    suit_str = card.__str__()[1]
    suitList.append(suit_str)
    
print(holeCards, rankList, suitList)
#checkAceSuited(rankList,suitList)
checkConnectedCards(rankList)

['9♣', 'J♥', 'J♦', '2♠'] ['9', 'J', 'J', '2'] ['♣', '♥', '♦', '♠']
9
0 7 8
T


False

In [107]:
# optimal strat case statement. Need to turn this into an actual executable method (change prints to returns)

# When to Raise
if (checkDoubleSuited(suitList)):
    for elem in top30HandsList:
        if(set(rankList) == set(elem)):
            print(obs["max_raise"])
    for elem in kkDoubleSuitedList:
        if(set(rankList) == set(elem)):
            print(obs["max_raise"])
    for elem in highFourInARowList:
        if(set(rankList) == set(elem)):
            print(obs["max_raise"])
    
elif(checkSuited(suitList) == True):
    for elem in akxxs:
        if(set(rankList) == set(elem)):
            print(obs["max_raise"])
            
# When to Check/"Limp"
elif(checkAceSuited(rankList,suitList)):
    for elem in aqxxs:
        if(set(rankList) == set(elem)):
            print(obs["call"])
            
    if(checkConnectedCards(rankList)):
        for elem in axxxs:
            if(set(rankList) == set(elem)):
                print(obs["call"])
    
elif(checkAceSuited(rankList,suitList)==False and checkDoubleSuited(suitList)==False and checkSuited(suitList)==False):
    for elem in lowFourInARowList:
        if(set(rankList) == set(elem)):
            print(obs["call"])
    
else: 
    alwaysFold(self)

#testing out components of above functions
a = ['A','A','K','K']
b = ['A','A','K','K', 'Q']

if (set(a) == set(b)):
    print('y')
else:
    print('n')


In [75]:
top30HandsList = [ ['A','A','K','K'], ['A','A','J','T'], ['A','A','Q','Q'], ['A','A','J','J'], ['A','A','T','T'],
                   ['A','A','9','9'], ['J','T','9','8'],
                   ['K','K','Q','Q'], ['K','K','J','J'], ['K','Q','J','T'], ['K','K','T','T'], ['K','K','A','Q'],
                   ['K','K','A','J'], ['K','K','A','T'], ['K','K','Q','J'], ['K','K','Q','T'], ['K','K','J','T'],
                   ['Q','Q','J','J'], ['Q','Q','T','T'], ['Q','Q','A','K'], ['Q','Q','A','J'], ['Q','Q','A','T'],
                   ['Q','Q','K','J'], ['Q','Q','K','T'], ['Q','Q','J','T'], ['Q','Q','J','9'], ['Q','Q','9','9'],
                   ['J','J','T','T'], ['J','J','T','9'] 
                 ]

high_face = ["T", "J", "Q", "K", "A"]
face = ["2","3","4","5","6","7","8","9", "T","J","Q","K","A"]

for i in range(len(face)):
    for j in range(len(face)):
        top30HandsList.append((["A", "A", face[i], face[j]]))

akxxs = []
for i in range(len(high_face)):
     for j in range(len(face)):
             akxxs.append((["A", "K", high_face[i], face[j]]))  
                
                
highFourInARowList = [ ['5','6','7','8'], ['6','7','8','9'], ['7','8','9','T'], ['8','9','T','J'], ['9','T','J','Q'],
                   ['T','J','Q','K'], ['J','Q','K','A']
                 ]

lowFourInARowList = [ ['4','5','6','7'], ['5','6','7','8'], ['6','7','8','9'], ['7','8','9','T'], ['8','9','T','J'], 
                     ['9','T','J','Q'],['T','J','Q','K'], ['J','Q','K','A'] ]

kkDoubleSuitedList = []
for i in range(len(face)):
    for j in range(len(face)):
        kkDoubleSuitedList.append((["K", "K", face[i], face[j]]))
        
        
# Hole Cards to Flop With
aqxxs = []
for i in range(len(high_face)):
     for j in range(len(face)):
             akxxs.append((["A", "Q", high_face[i], face[j]]))
                
axxxs = []
for i in range(len(face)):
    for j in range(len(face)):
        for k in range(len(face)):
            axxxs.append((["A",face[i],face[j],face[k]]))

# suited as long as there are at least 2 of any one suit 
def checkSuited(self, suitList):
    spadeCount = suitList.count(chr(9824)) 
    heartCount = suitList.count(chr(9829))
    diamondCount = suitList.count(chr(9830))
    clubCount = suitList.count(chr(9827))
    
    if(spadeCount or heartCount or diamondCount or clubCount >= 2):
        return True
    else: 
        return False
    
# specifically check that if the hole cards contain an Ace, it is suited
def checkAceSuited(self, rankList,suitList):
    rank_set = set(rankList) 
    
    ace_pos_list = [ i for i in range(len(rankList)) if rankList[i] == 'A' ]
    #print(ace_pos_list)
    
    if 'A' in rank_set: 
        ace_suit_list = [ suitList[i] for i in range(len(rankList)) if rankList[i] == 'A' ]
        
        for elem in ace_suit_list:
            #print(elem)
            if (suitList.count(elem) >= 2):
                return True
        
    else: 
         return True
    
    
# double suited if no suit counts are zero or odd and sum of suit counts == 4
def checkDoubleSuited(self, suitList):
    spadeCount = suitList.count(chr(9824))
    heartCount = suitList.count(chr(9829))
    diamondCount = suitList.count(chr(9830))
    clubCount = suitList.count(chr(9827))
    
    if(
    
        2 in {spadeCount, heartCount, diamondCount, clubCount} and ((spadeCount + heartCount == 4) or 
         (spadeCount + diamondCount == 4) or (spadeCount + clubCount == 4) or
        (heartCount + diamondCount == 4) or (heartCount + clubCount == 4) or (diamondCount + clubCount == 4))
        
    ):
        return True
    else:
        return False

# check if at least 2 cards from the hole cards are consecutive
def checkConnectedCards(self, rankList):
    for elem in rankList:
        idx = rankList.index(elem)
        face_idx = face.index(elem)
        
        if(face[face_idx + 1] in rankList):
            return True
            
        else:
            return False

# fold observation
def alwaysFold(self):
    return -1

    
# optimal strat
def optimalStrat(self, obs):

    holeCards = obs["hole_cards"]

    rankList = []
    suitList = []

    for card in holeCards:
        rank_str = card.__str__()[0]
        rankList.append(rank_str)
    
        suit_str = card.__str__()[1]
        suitList.append(suit_str)
    
    # When to Raise
    if (checkDoubleSuited(suitList) == True):
        for elem in top30HandsList:
            if(set(rankList) == set(elem)):
                return(obs["max_raise"])
        for elem in kkDoubleSuitedList:
            if(set(rankList) == set(elem)):
                return(obs["max_raise"])
        for elem in highFourInARowList:
            if(set(rankList) == set(elem)):
                return(obs["max_raise"])
    
    elif(checkSuited(suitList) == True):
        for elem in akxxs:
            if(set(rankList) == set(elem)):
                return(obs["max_raise"])
            
    # When to"Limp"
    elif(checkAceSuited(rankList,suitList)):
        for elem in aqxxs:
            if(set(rankList) == set(elem)):
                return(obs["call"])
            
        if(checkConnectedCards(rankList)):
            for elem in axxxs:
                if(set(rankList) == set(elem)):
                    return(obs["call"])
    
    elif(checkAceSuited(rankList,suitList)==False and checkDoubleSuited(suitList)==False and checkSuited(suitList)==False):
        for elem in lowFourInARowList:
            if(set(rankList) == set(elem)):
                return(obs["call"])
    
    else: 
        alwaysFold(self)
